In [23]:
import os

import pandas as pd
import s3fs
import zipfile
from scipy.io import loadmat

## Download les données

In [17]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [18]:
# Lister les challenges
fs.ls("gvimont/diffusion/hackathon-minarm-2024")

['gvimont/diffusion/hackathon-minarm-2024/AIVSAI',
 'gvimont/diffusion/hackathon-minarm-2024/Acoustique',
 'gvimont/diffusion/hackathon-minarm-2024/Similarité']

In [19]:
# Lister les fichiers d'un challenge
fs.ls("gvimont/diffusion/hackathon-minarm-2024/Similarité")

['gvimont/diffusion/hackathon-minarm-2024/Similarité/.keep',
 'gvimont/diffusion/hackathon-minarm-2024/Similarité/archive.zip']

In [20]:
# Télécharger les données dans le service
PATH_IN = 'gvimont/diffusion/hackathon-minarm-2024/Similarité/archive.zip'
fs.download(PATH_IN, 'data/archive.zip')

[None]

In [21]:
# Décompresser les données
with zipfile.ZipFile("data/archive.zip","r") as zip_file:
    zip_file.extractall("data/")

## Create dataframe

In [45]:
mat_data = loadmat("data/cars_annos.mat")

annotations = mat_data["annotations"]

data = []

for ann in annotations[0]:
    relative_im_path = ann['relative_im_path'][0]
    bbox_x1 = ann['bbox_x1'][0][0]
    bbox_y1 = ann['bbox_y1'][0][0]
    bbox_x2 = ann['bbox_x2'][0][0]
    bbox_y2 = ann['bbox_y2'][0][0]
    class_name = ann['class'][0][0]
    test = ann['test'][0][0]
    data.append([relative_im_path, bbox_x1, bbox_y1, bbox_x2, bbox_y2, class_name, test])

df = pd.DataFrame(data, columns=['relative_im_path', 'bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class', 'test'])
df.head

<bound method NDFrame.head of          relative_im_path  bbox_x1  bbox_y1  bbox_x2  bbox_y2  class  test
0      car_ims/000001.jpg      112        7      853      717      1     0
1      car_ims/000002.jpg       48       24      441      202      1     0
2      car_ims/000003.jpg        7        4      277      180      1     0
3      car_ims/000004.jpg       33       50      197      150      1     0
4      car_ims/000005.jpg        5        8       83       58      1     0
...                   ...      ...      ...      ...      ...    ...   ...
16180  car_ims/016181.jpg       38       36      375      234    196     1
16181  car_ims/016182.jpg       29       34      235      164    196     1
16182  car_ims/016183.jpg       25       32      587      359    196     1
16183  car_ims/016184.jpg       56       60      208      186    196     1
16184  car_ims/016185.jpg        1        1      200      131    196     1

[16185 rows x 7 columns]>